In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

from tensorflow import keras

#import tensorflow_io as tfio

import time

from functions_dataCreation import *
from functions_modelArchitectures import *
from class_LRFinder import *


from tensorflow.keras.callbacks import Callback

import pandas as pd

import wandb
from wandb.keras import WandbCallback

IMG_SIZE = 128

METRICS = [
    #   keras.metrics.TruePositives(name='tp'),
    #   keras.metrics.FalsePositives(name='fp'),
    #   keras.metrics.TrueNegatives(name='tn'),
    #   keras.metrics.FalseNegatives(name='fn'), 
    #   keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]


1 Physical GPUs, 1 Logical GPUs


In [2]:
train = createIODataset(140,'../data/Train')
test = createIODataset(4,'../data/Test')


In [3]:
# # Shuffle the elements of the dataset.
# train = train.shuffle(buffer_size=10240,reshuffle_each_iteration=True)

# # By default image data is uint8, so convert to float32 using map().
# # my_train = my_train.map(lambda x, y: (tf.image.convert_image_dtype(x, tf.float32), y))

# # prepare batches the data just like any other tf.data.Dataset
# train = train.batch(256,drop_remainder=True).prefetch(10)
# # train = train.repeat()

# test = test.shuffle(buffer_size=2048,reshuffle_each_iteration=True).batch(256,drop_remainder=True).prefetch(10)
# # test = test.repeat()


In [4]:
train = train.repeat(-1)
# train = train.shuffle(buffer_size=10240*20,reshuffle_each_iteration=True)
# train = train.repeat(5)
train = train.batch(256,drop_remainder=True)
train = train.prefetch(10)

test = test.repeat(-1)
# test = test.shuffle(buffer_size=10240,reshuffle_each_iteration=True)
# train = train.repeat(5)
test = test.batch(1024,drop_remainder=True)
test = test.prefetch(10)



## Faster LR
 
* Train a CNN for 10000 epochs with LR of 1e-4
* Changed glorot_normal for relu activation
* Added BN for first image layer

For Batch Size of 256, to complete 1 epoch the model take 2114 steps. So total dataset size is approx 500K images. Time taken per epoch is 198s = 3.5mins

Lets do 200 steps per epoch i.e 51200 (samples) ie 10% of total per epoch




In [5]:

wandb.init(project="candlestick-CNN", name = 'Deeper tfData 1e-7 100steps per epoch shuffle size 200K Deeper CNN NO Shuffle' )

modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'

modelBase['convLayerMultiplier'] = 1

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_Dropout = modelBase.copy()
modelBase_with_Dropout['name'] = 'modelBase_with_Dropout'
modelBase_with_Dropout['batchnormalization'] = False
# modelBase_with_Dropout['dropout'] = 0.00001

modelBase_with_Dropout['kernelSize'] = (3,3)
modelBase_with_Dropout['filters'] = [10,15,20,25,30,35,40,45,50,55,60]
modelBase_with_Dropout['poolSize'] = (6,6)

lr = 1e-7

df = pd.DataFrame()

start_time = time.time()

checkpoint_path = '../data/callbacks/'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=500)   

model = createCNN(modelBase_with_Dropout)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr)
        ,loss=tf.keras.losses.binary_crossentropy
        ,metrics=[METRICS])

history = model.fit(train
                #,batch_size = 128
                ,epochs=5000
                ,steps_per_epoch=100
                ,verbose=0
                ,validation_data=test                
                ,validation_freq = 100
                ,validation_steps = 10
                ,callbacks=[WandbCallback(),cp_callback]
                )

# temp = pd.DataFrame(history.history).rename_axis("epoch")
# temp['elapsed'] = round((time.time() - start_time)/60,2)
# var_params = "Deeper_" + "_LR_"  + str(lr)
# temp['params'] = var_params

# print("Elapsed time " + str(round((time.time() - start_time)/60,2)) + var_params)

# model.save('../data/savedmodels') 



Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: amitagni (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



Epoch 00500: saving model to ../data/callbacks/

Epoch 01000: saving model to ../data/callbacks/

Epoch 01500: saving model to ../data/callbacks/

Epoch 02000: saving model to ../data/callbacks/

Epoch 02500: saving model to ../data/callbacks/

Epoch 03000: saving model to ../data/callbacks/

Epoch 03500: saving model to ../data/callbacks/

Epoch 04000: saving model to ../data/callbacks/

Epoch 04500: saving model to ../data/callbacks/

Epoch 05000: saving model to ../data/callbacks/


In [6]:
1024*100*6

614400